In [9]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [10]:
import mtt
from os.path import join, realpath, split
from os import getcwd
sbfile = join(realpath(getcwd()),'cascade-reversible.sb')
wiring = mtt.Wiring.fromFile(sbfile)
wiring

In [11]:
model = mtt.MTT(wiring)
model.draw(3)

# Digitized current config

In [12]:
for block in wiring.blocks:
    print(block.getDigitizedParameterString(model))

# Simulink Export

In [13]:
model.exportToSimulink(join(realpath(getcwd()),'simulink','cascade_reversible'), 'cascade_reversible', 3)

# Block Simulation

In [14]:
# block simulation
model.reset()
for k in range(3):
    print('block {}: rate_fw = {}'.format(k,model.blocks[k].rate_fw))
    print('block {}: rate = {}'.format(k,model.blocks[k].rate))
mtt_results = model.simulate(0,1,100)
model.plot(xtitle='time(s)', ytitle='conc.(nA)')

# SBML Simulation

In [15]:
# SBML comparison
import tellurium as te
with open(sbfile) as f:
    rr_model = te.loada(f.read())
rr_results = rr_model.simulate(0,1,1000)
rr_model.plot(xtitle='time(s)', ytitle='conc.(nA)')

In [16]:
nrows = 2
ncols = 1
fig,axes = plt.subplots(nrows, ncols, sharex='col', sharey='row', figsize=(6,8))

col = 0
for row,(r,title) in enumerate(zip((mtt_results,rr_results),('Block Simulation','SBML Simulation'))):
    a = axes[row]
    a.set_title(title, fontsize=14)
    if row == nrows-1:
        a.set_xlabel('time(s)', fontsize=14)
    a.set_ylabel('conc.(nA)', fontsize=14)
    for k in range(1,r.shape[1]):
        a.plot(r[:,0], r[:,k], label=r.colnames[k], color='C'+str(1+k), linewidth=2)
    handles, labels = a.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower right')

pdfname = split(sbfile)[1].split('.')[0]+'.pdf'
plt.savefig(pdfname, format='pdf')